In [9]:
import imageio
imageio.plugins.ffmpeg.download()

In [38]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [30]:
#Canny Edge Detection
def getEdges(gray_img):
    # Define Canny Parameters
    kernel = 5 #blurs image to remove noisy lines
    lo = 50 #lower bound for connection to high gradient (0-hi)
    hi = 150 #lower bound to be considered high gradient (0-255)
        
    gray_blur = cv2.GaussianBlur(gray_img,(kernel,kernel),0)
    edges = cv2.Canny(gray_blur,lo,hi)
    
    return edges

#Detect Lines 
def getBasicLines(mask_img,edge_img):
    
    
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = 1* np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 8   # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 8 #minimum number of pixels making up a line
    max_line_gap = 8  # maximum gap in pixels between connectable line segments
    
    
    color_edges = np.dstack((edge_img, edge_img, edge_img)) 
    
    # create blank image to draw on
    blank_image = np.copy(mask_img)*0
    blank_image = np.dstack((blank_image,blank_image,blank_image))
    
    lines = cv2.HoughLinesP(mask_img, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)
    
    
    return lines

def avgPoints(points):
    
    ret = [0 for i in range(2)]
    for i in range(2):
        ret[i] = sum(point[i] for point in points)
    
    for i in range(2):
        ret[i] /= len(points)
    return ret

def calcYIntercept(m,point):
    return point[1] - point[0]*m

def calcX(y,m,b):
    return int((y-b)/m)

def calcEndpoints(m,point,y1,y2):
    b = calcYIntercept(m,point)
    #print("B: ",b)
    x1 = calcX(y1,m,b)
    x2 = calcX(y2,m,b)
    return [x1,y1,x2,y2]
    

def combineLines(slopes,centers):
    
    if len(slopes) != 0:
        main_slope = sum(slopes)/len(slopes)
        main_center = avgPoints(centers)
        main_endpoints = calcEndpoints(main_slope,main_center,360,720)
    else:
        main_endpoints = [0,0,0,0]
        
    return main_endpoints
    
    
        
#Calculates Main Left / Right lines   
def calcMainLines(blank_image,lines):
    
    left_slopes = []
    left_centers = []
    right_slopes = []
    right_centers = []
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            if x1-x2 != 0:
                
                #Calc Line Parameters
                slope = (y1-y2)/(x1-x2)
                center = [(x1+x2)/2,(y1+y2)/2]
                abs_slope = abs(slope)
                
                #Filter lines too vertical / horizontal
                if abs_slope > 3/7 and abs_slope < 6/7:
                    
                    #If the line has a positive slope, assume left slope
                    if slope == abs_slope:
                        left_slopes.append(slope)
                        left_centers.append(center)
                        
                    else:
                        right_slopes.append(slope)
                        right_centers.append(center)
      
    # calc and draw left line
    left_endpoints = combineLines(left_slopes,left_centers)
    cv2.line(blank_image,(left_endpoints[0],left_endpoints[1]),(left_endpoints[2],left_endpoints[3]),(255,0,0),10)
    
    # calc and draw right line
    right_endpoints = combineLines(right_slopes,right_centers)
    cv2.line(blank_image,(right_endpoints[0],right_endpoints[1]),(right_endpoints[2],right_endpoints[3]),(255,0,0),10)

    # not blank unless there were no lines
    return blank_image
def overlayLines(img,line_image):
    return cv2.addWeighted(img,0.8,line_image,1,0)

#Applies a mask to the image so only the road is visible
def maskImg(img,vertices):
    
    ignore_mask_color = 255
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, np.int32(vertices), ignore_mask_color)
    masked_img = cv2.bitwise_and(img, mask)
    
    return masked_img

#Convert Grayscale Image To Be Compatable With RGB
def grayToThreeChannel(img):
    shape = img.shape
    ret = np.zeros([shape[0],shape[1],3])
    
    # copys grayscale image to a 3 channel image
    for i in range(0,3):
        ret[..., i] = img
        
    return ret

In [32]:
import os

files = os.listdir("test_videos/")
for file in files:
    print(file)
    if file[0] != "." and file[0:6] != "output":
        new_clip_output = "test_videos/output_"+file
        test_clip = VideoFileClip("test_videos/"+file)
        new_clip = test_clip.fl_image(rgb2Lanes) 
        %time new_clip.write_videofile(new_clip_output, audio=False)

.DS_Store
challenge.mp4
[MoviePy] >>>> Building video test_videos/output_challenge.mp4
[MoviePy] Writing video test_videos/output_challenge.mp4


100%|██████████| 251/251 [00:23<00:00, 10.68it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos/output_challenge.mp4 

CPU times: user 9.92 s, sys: 2.92 s, total: 12.8 s
Wall time: 25.4 s
output_challenge.mp4
output_solidWhiteRight.mp4
output_solidYellowLeft.mp4
solidWhiteRight.mp4
[MoviePy] >>>> Building video test_videos/output_solidWhiteRight.mp4
[MoviePy] Writing video test_videos/output_solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:08<00:00, 29.07it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos/output_solidWhiteRight.mp4 

CPU times: user 4.17 s, sys: 1.15 s, total: 5.33 s
Wall time: 9.29 s
solidYellowLeft.mp4
[MoviePy] >>>> Building video test_videos/output_solidYellowLeft.mp4
[MoviePy] Writing video test_videos/output_solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:27<00:00, 24.87it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos/output_solidYellowLeft.mp4 

CPU times: user 13.3 s, sys: 3.83 s, total: 17.2 s
Wall time: 28.3 s


In [33]:
HTML("""
<video width="640" height="300" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(new_clip_output))

In [45]:
files = os.listdir("test_images/")
for file in files:
    print(file)
    if file[0] != "." and file[0:6] != "output":
        img = mpimg.imread("test_images/"+file)
        lanes_img = rgb2Lanes(img)
        plt.imshow(lanes_img)
        cv2.imwrite("test_images/output_"+file,lanes_img)

output_solidWhiteCurve.jpg
output_solidWhiteRight.jpg
output_solidYellowCurve.jpg
output_solidYellowCurve2.jpg
output_solidYellowLeft.jpg
output_whiteCarLaneSwitch.jpg
solidWhiteCurve.jpg
solidWhiteRight.jpg
solidYellowCurve.jpg
solidYellowCurve2.jpg
solidYellowLeft.jpg
whiteCarLaneSwitch.jpg


In [53]:
img = mpimg.imread("test_images/solidWhiteRight.jpg")

gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
cv2.imwrite("pipeline_images/grayscale.jpg",grayToThreeChannel(gray))

edges = getEdges(gray)
cv2.imwrite("pipeline_images/edges.jpg",grayToThreeChannel(edges))

imshape = img.shape[:2][::-1]
general_vertices = np.array([[[.08,1],[.45,.6],[.55,.6],[.92,1]]])
vertices = np.int32(np.multiply(general_vertices ,np.tile(imshape,(4,1))))

masked_edges = maskImg(edges,vertices)
cv2.imwrite("pipeline_images/masked_edges.jpg",grayToThreeChannel(masked_edges))

lines = getBasicLines(masked_edges,edges)
major_lines = calcMainLines(img*0,lines)
cv2.imwrite("pipeline_images/major_lines.jpg",major_lines)

lines_overlayed = overlayLines(img,major_lines)
cv2.imwrite("pipeline_images/lines_overlayed.jpg",lines_overlayed)

True

In [31]:
#Find Main Lane Lines from Color Image
def rgb2Lanes(img):

    
    # Three Channel Color -> One Channel Gray
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    
    # Get Edges 
    edges = getEdges(gray)
    
    # Define parameters for mask
    imshape = img.shape[:2][::-1]
    general_vertices = np.array([[[.08,1],[.45,.6],[.55,.6],[.92,1]]])
    vertices = np.int32(np.multiply(general_vertices ,np.tile(imshape,(4,1))))

    # Apply Mask
    masked_edges = maskImg(edges,vertices)
    
    # Find lines using Hough Transform
    lines = getBasicLines(masked_edges,edges)
    
    # Combine lines to find major lines
    major_lines = calcMainLines(img*0,lines)
    
    # Combine line image with original image
    lines_overlayed = overlayLines(img,major_lines)
    return lines_overlayed